# Solving the Diet Problem for Armenia's Five Markets

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [2]:
NUTRITION_FILE = '../data/nutrition.csv'
PRICING_FILE = '../data/pricing.csv'

# If a particular market does not have a food, we set
# its price to be "infinity" (i.e. 999999). np.nan
# does not suffice, as we must have float datatypes.
INFINITY = 999999

In [3]:
# Read in data csvs
nutrition = pd.read_csv(NUTRITION_FILE, index_col=0)
pricing = pd.read_csv(PRICING_FILE, index_col=0)

In [4]:
# Group Armenia data by country, locality and market, in that order.
grouped = (pricing[pricing.country_name == 'Armenia']
                  .groupby(['country_name', 'locality_name', 'market_name']))

In [5]:
solns = {}

# Solve diet problem for all groups
# (i.e. all markets in all localities in Armenia)
for market, idx in grouped.groups.items():
    df = pricing.loc[idx]

    # Form A matrix (nutritional values of each food) and
    # b vector (dietary requirements for each nutrient).
    # Dietary requirements: protein, fat, carbs, fiber, in that order.
    # From http://www.netrition.com/rdi_page.html
    A_ub = -np.transpose(nutrition.values)
    b_ub = -np.array([50, 65, 300, 25])

    # Construct c vector appropriately (i.e. add INFINITYs to
    # the appropriate foods)
    c = pd.Series(data=INFINITY*np.ones(84), index=nutrition.index)
    c.loc[df.commodity_name] = df.price.values
    c = c.values

    # Solve LP using the default simplex algorithm.
    solns[market] = linprog(c, A_ub, b_ub)

In [6]:
# Collect optimization status, minimum value and minimum into one array
data = np.hstack([
    np.transpose(
        np.vstack([[soln.status for soln in solns.values()],
                   [soln.fun for soln in solns.values()]])
    ),
    [soln.x for soln in solns.values()]
])

In [7]:
# Cast array in a dataframe and save to a csv.
df = pd.DataFrame(data=data,
                  index=solns.keys(),
                  columns=['status', 'fun'] + nutrition.index.tolist())
df.to_csv('armenia.csv')

## Sensitivity Analysis, Part 2

Trying it this time using the dual problem...

In [27]:
duals = {}

# Solve diet problem for all groups
# (i.e. all markets in all localities in Armenia)
for market, idx in grouped.groups.items():
    df = pricing.loc[idx]

    # Form A matrix (nutritional values of each food) and
    # b vector (dietary requirements for each nutrient).
    # Dietary requirements: protein, fat, carbs, fiber, in that order.
    # From http://www.netrition.com/rdi_page.html
    A_ub = nutrition.values
    b_ub = np.array([50, 65, 300, 25])

    # Construct c vector appropriately (i.e. add INFINITYs to
    # the appropriate foods)
    c = pd.Series(data=INFINITY*np.ones(84), index=nutrition.index)
    c.loc[df.commodity_name] = df.price.values
    c = c.values

    # Solve LP using the default simplex algorithm.
    duals[market] = linprog(b_ub, A_ub, c)

In [28]:
[dual.x for dual in duals.values()]

[array([0., 0., 0., 0.]),
 array([0., 0., 0., 0.]),
 array([0., 0., 0., 0.]),
 array([0., 0., 0., 0.]),
 array([0., 0., 0., 0.])]

## Sensitivity Analysis

Here, we focus our attention on the market of Armavir, in the Lori locality, in Armenia.

In [28]:
x = df.iloc[0].drop(['status', 'fun'])

In [29]:
# Only eggs and pasta are demanded.
demanded = x[x != 0].index

In [30]:
mask = (pricing.country_name == 'Armenia') & (pricing.locality_name == 'Lori') & (pricing.market_name == 'Armavir')
pricing = pricing[mask].drop(['country_name', 'locality_name', 'market_name'], axis=1).reset_index(drop=True)

In [31]:
perturbed_solns = {}
perturbations = [1.1, 1.5, 0.9, 0.5]  # Increase and decrease by 10% and 50%

# First, perturb nutrition data
for perturbation in perturbations:
    for food in demanded:
        perturbed_nutrition = nutrition.copy()
        perturbed_nutrition.loc[food] *= perturbation
        
        A_ub = -np.transpose(perturbed_nutrition.values)
        b_ub = -np.array([65, 300, 25, 50])

        # Construct c appropriately (i.e. add 0s to the appropriate foods)
        c = pd.Series(data=INFINITY*np.ones(84), index=perturbed_nutrition.index)
        c.loc[pricing.commodity_name] = pricing.price.values
        c = c.values

        perturbed_solns[('nutrition', food, perturbation)] = linprog(c, A_ub, b_ub)
        
# Next, perturb pricing data
for perturbation in perturbations:
    for food in demanded:
        perturbed_pricing = pricing.copy()
        perturbed_pricing.loc[perturbed_pricing.commodity_name == food, 'price'] *= perturbation
        
        A_ub = -np.transpose(nutrition.values)
        b_ub = -np.array([65, 300, 25, 50])

        # Construct c appropriately (i.e. add 0s to the appropriate foods)
        c = pd.Series(data=INFINITY*np.ones(84), index=nutrition.index)
        c.loc[perturbed_pricing.commodity_name] = perturbed_pricing.price.values
        c = c.values

        perturbed_solns[('pricing', food, perturbation)] = linprog(c, A_ub, b_ub)

In [32]:
# Collect optimization status, minimum value and minimum into one array
data = np.hstack([
    np.transpose(
        np.vstack([[soln.status for soln in perturbed_solns.values()],
                   [soln.fun for soln in perturbed_solns.values()]])
    ),
    [soln.x for soln in perturbed_solns.values()]
])

In [33]:
data.shape

(16, 86)

In [34]:
df = pd.DataFrame(data=data,
                  index=perturbed_solns.keys(),
                  columns=['status', 'fun'] + nutrition.index.tolist())

In [38]:
df.loc[:, df.sum() != 0]

fun      Eggs  Lentils     Pasta
nutrition Eggs  1.1  272.752867  0.600422  0.00000  0.561798
          Pasta 1.1  257.127235  0.660464  0.00000  0.510725
          Eggs  1.5  259.943863  0.440310  0.00000  0.561798
          Pasta 1.5  202.649877  0.660464  0.00000  0.374532
          Eggs  0.9  283.427037  0.733849  0.00000  0.561798
          Pasta 0.9  294.421263  0.672088  0.46729  0.000000
          Eggs  0.5  330.393386  1.320929  0.00000  0.561798
          Pasta 0.5  294.421263  0.672088  0.46729  0.000000
pricing   Eggs  1.1  282.839958  0.660464  0.00000  0.561798
          Pasta 1.1  294.421263  0.672088  0.46729  0.000000
          Eggs  1.5  303.974815  0.660464  0.00000  0.561798
          Pasta 1.5  294.421263  0.672088  0.46729  0.000000
          Eggs  0.9  272.272529  0.660464  0.00000  0.561798
          Pasta 0.9  255.084334  0.660464  0.00000  0.561798
          Eggs  0.5  251.137672  0.660464  0.00000  0.561798
          Pasta 0.5  165.196693  0.660464  0.00000  0.561798